In [27]:
!python -m spacy download pt_core_news_sm
!pip install enelvo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 109.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [28]:
from google.colab import drive
from enelvo.normaliser import Normaliser
import re
import pandas as pd
import spacy
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
nlp_pt = spacy.load("pt_core_news_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


# Pré-processamento

## 0) Base de Dados

In [31]:
norm = Normaliser(tokenizer='readable')
chats_clients = pd.read_csv('/content/drive/Shareddrives/grupo3moshi/chats_clients.csv')
chats_clients.head()

,No,Intencao,Pergunta,Resposta
0,1,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d..."
1,2,Como fazer remessa,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f..."
2,3,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ..."
3,4,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...
4,5,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...


In [32]:
chats_clients.fillna('erro ao processar a pergunta', inplace=True)

### 0.1) Correção de gramática e erros de digitação

Existem algumas palavras escritas erradas e erros de digitação encontrados durante a análise exploratória. Por isso, foi necessário essa fase de correção de gramática e erros de digitação.

In [33]:
def correct_grammar(text):
  return norm.normalise(text)

Resultado Esperado

```
como faço depósito no japão
boa tarde
quanto tempo levará para o beneficiário receber o dinheiro no brasil ?
```

In [34]:
texts_with_errors = ["Como fasso depósito no Japão", "Boaa tarde", "Quanto tempo levará para o beneficiário receber o dinhero no Brasil?"]

for i in range(len(texts_with_errors)):
    print(f"Frase {i}: {correct_grammar(texts_with_errors[i])}")

Frase 0: como faço depósito no japão
Frase 1: boa tarde
Frase 2: quanto tempo levará para o beneficiário receber o dinheiro no brasil ?


## 1) Reconhecimento de Nomes de Entidades

Mais informações sobre a implementação e a técnica podem ser obtidas no [Notebook Python da implementação do NER](./NER_PreProcessamento.ipynb)

In [35]:
def extract_ents(text):
    # Verify if it is a valid text
    if not isinstance(text, str):
        return []

    #  Use the portuguese model to process the text
    doc_pt = nlp_pt(text)

    # Place named entities in a dictionary
    ents = [{f'{ent.text}': ent.label_} for ent in doc_pt.ents]
    return ents

Resultado Esperado

```
[{'japão': 'LOC'}]
[]
[{'brasil': 'LOC'}]
```

In [36]:
corrected_texts = [correct_grammar(text) for text in texts_with_errors]

for text in corrected_texts:
    result = extract_ents(text)
    print(f"Texto: {text}, Entidades: {result}")

Texto: como faço depósito no japão, Entidades: [{'japão': 'LOC'}]
Texto: boa tarde, Entidades: []
Texto: quanto tempo levará para o beneficiário receber o dinheiro no brasil ?, Entidades: [{'brasil': 'LOC'}]


## 2) Tokenização

A Tokenização consiste no processo de divisão de um texto em partes menores, tokens, sejam elas palavras, caracteres ou até frases, de forma que o modelo de LLM possa processar e manipular os tokens com os algoritmos. Esse processo é a primeira etapa do treinamento do modelo e essencial para o resultado das outras etapas. ([Fonte](https://learn.microsoft.com/pt-br/dotnet/ai/conceptual/understanding-tokens))


In [37]:
def tokenize(text):

  doc = nlp_pt(text)
  tokens = [token.text for token in doc if not token.is_punct]
  return tokens

Resultado Esperado

```
['como', 'faço', 'depósito', 'no', 'japão']
['boa', 'tarde']
['quanto', 'tempo', 'levará', 'para', 'o', 'beneficiário', 'receber', 'o', 'dinheiro', 'no', 'brasil']
```

In [38]:
tokens = []
for text in corrected_texts:
    result = tokenize(text)
    tokens.append(result)
    print(f"Texto: {text}, Tokenizado: {result}")

Texto: como faço depósito no japão, Tokenizado: ['como', 'faço', 'depósito', 'no', 'japão']
Texto: boa tarde, Tokenizado: ['boa', 'tarde']
Texto: quanto tempo levará para o beneficiário receber o dinheiro no brasil ?, Tokenizado: ['quanto', 'tempo', 'levará', 'para', 'o', 'beneficiário', 'receber', 'o', 'dinheiro', 'no', 'brasil']


## 3) Remoção de Stop Words


A etapa de remoção de Stop Words é um processo de refinamento e eliminação de palavras irrelevantes para o processamento do modelo de LLM, ao consultar em uma lista de palavras irrelevantes no idioma em questão. Assim, a dimensionalidade do problema é diminuída e a eficiência do modelo aumentada com a eliminação de palavras irrelevantes, como "a", "e", "de". ([Fonte](https://fineproxy.org/pt/wiki/stopword-removal/#:~:text=A%20remo%C3%A7%C3%A3o%20de%20palavras%20de,automatizado%20e%20classifica%C3%A7%C3%A3o%20de%20textos.))

Mais informações sobre a implementação e a técnica podem ser obtidas no [Notebook Python - Stop Words](./StopWords.ipynb)

In [39]:
stop_words = set(stopwords.words('portuguese'))
def remover_stop_words(tokens):
  filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
  return filtered_tokens

Resultado Esperado

```
['faço', 'depósito', 'japão']
['boa', 'tarde']
['quanto', 'tempo', 'levará', 'beneficiário', 'receber', 'dinheiro', 'brasil']
```

In [40]:
tokens_sem_stop_words = []
for token in tokens:
    sem_stop_words = remover_stop_words(token)
    tokens_sem_stop_words.append(sem_stop_words)
    print(f"Tokens com Stop Words: {token}, Sem Stop Words: {sem_stop_words}")

Tokens com Stop Words: ['como', 'faço', 'depósito', 'no', 'japão'], Sem Stop Words: ['faço', 'depósito', 'japão']
Tokens com Stop Words: ['boa', 'tarde'], Sem Stop Words: ['boa', 'tarde']
Tokens com Stop Words: ['quanto', 'tempo', 'levará', 'para', 'o', 'beneficiário', 'receber', 'o', 'dinheiro', 'no', 'brasil'], Sem Stop Words: ['quanto', 'tempo', 'levará', 'beneficiário', 'receber', 'dinheiro', 'brasil']


## 4) Lematização

A Lematização é o processo de extrair o infinitivo dos verbos, de forma que haja uma padronização na utilização das palavras. Isso é importante para a rede reconhecer de forma mais simples os padrões entre os tokens. ([Fonte](https://blog.bitso.com/pt-br/criptomoedas/diferenca-entre-tokenizacao-e-lematizacao#:~:text=Lematiza%C3%A7%C3%A3o%20%C3%A9%20um%20tipo%20de,fim%20de%20simplificar%20a%20palavra.))

Mais informações sobre a implementação e a técnica podem ser obtidas no [Notebook Python - Lematização](./Lematização_e_Steeming_Sprint_1_Pedro_Munhoz.ipynb)

In [41]:
def lemmatize(text):
    text = ' '.join(text)
    doc = nlp_pt(text)
    lemmatized_token = [token.lemma_ for token in doc]

    return lemmatized_token

Resultado Esperado

```
['fazer', 'depósito', 'Japão']
['bom', 'tarde']
['quanto', 'tempo', 'levar', 'beneficiário', 'receber', 'dinheiro', 'Brasil']

```



In [42]:
for i in range(len(tokens_sem_stop_words)):
    tokens_lematizados = lemmatize(tokens_sem_stop_words[i])
    print(f"Frase {i}: {tokens_lematizados}")

Frase 0: ['fazer', 'depósito', 'Japão']
Frase 1: ['bom', 'tarde']
Frase 2: ['quanto', 'tempo', 'levar', 'beneficiário', 'receber', 'dinheiro', 'Brasil']


## 5) Pipeline completa

A fim de simplificar o pré-processamento, junta-se todos os processos empregados na seguite função até então (em desenvolvimento)

In [43]:
def pre_processamento(df):
  df['Pergunta'] = df['Pergunta'].apply(correct_grammar)
  df['Pergunta - NER'] = df['Pergunta'].apply(extract_ents)
  df['Pergunta'] = df['Pergunta'].apply(tokenize)
  df['Pergunta'] = df['Pergunta'].apply(remover_stop_words)
  df['Pergunta'] = df['Pergunta'].apply(lemmatize)
  df['Resposta'] = df['Resposta'].apply(correct_grammar)
  df['Resposta - NER'] = df['Resposta'].apply(extract_ents)
  df['Resposta'] = df['Resposta'].apply(tokenize)
  df['Resposta'] = df['Resposta'].apply(remover_stop_words)
  df['Resposta'] = df['Resposta'].apply(lemmatize)


### 5.1) Resultado Final

In [44]:
pre_processamento(chats_clients)
chats_clients.head()

ERROR:enelvo.normaliser:Failed to normalise word "paypayginkou"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "zayrucard"!
ERROR:enelvo.normaliser:Failed to normalise word "brastelremit"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "mitsuisumitomo"!
ERROR:enelvo.normaliser:Failed to normalise word "incomodarmuito"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "ゆうちょ銀行"!
ERROR:enelvo.normaliser:Failed to normalise word "horariohttps"!
ERROR:enelvo.normaliser:Failed to normalise word "brastelremit"!
ERROR:enelvo.normaliser:Failed to normalise word "horariohttps"!
ERROR:enelvo.normaliser:Failed to normalise word "brastelremit"!
ERROR:enelvo.normaliser:F

,No,Intencao,Pergunta,Resposta,Pergunta - NER,Resposta - NER
0,1,Como depositar,"[bom, dia, tudo, bem, gostar, saber, sobre, ca...","[bom, dia, sim, senhor, poder, utilizar, cartã...",[{'marta verde': 'LOC'}],"[{'bom dia !': 'MISC'}, {'jp bank': 'MISC'}]"
1,2,Como fazer remessa,"[enviar, dinheiro, Japão]","[inscrever, serviço, remessar, favor, baixe, a...",[{'do japão': 'LOC'}],[{'appplication': 'MISC'}]
2,3,Tempo de remessa,"[quanto, tempo, levar, beneficiário, receber, ...","[via, regro, remesso, pago, ver, pia, dentro, ...",[],[]
3,4,"Pedido de envio via metodo ""ByPhone""","[bom, tarde, acabar, fazer, transferência, 22,...","[ir, processar, solicitação, obrigado, ótimo, ...",[],[]
4,5,"Pedido de envio via metodo ""ByPhone""","[poder, fazer, remesso, 2, 2yenes, bbb, rer, o...","[ir, processar, solicitação, obrigado, ótimo, ...",[{'bbb': 'MISC'}],[]
